In [ ]:
import numpy as np
import torch

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import DatasetTemplate
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils

from pathlib import Path
import yaml, os, glob
import logging

yaml_path = "config.yaml"
with open(yaml_path, 'r') as f:
    para_cfg = yaml.load(f, Loader=yaml.FullLoader)

cfg_root = para_cfg['cfg_root']
model_path = para_cfg['model_path']
pcd_path = "/home/ackerman/Workspace/OpenPCDet_ErrorModel/000000.bin"
data_path = "/media/ackerman/Windows/Dataset/KITTI/object/training/"

results_path = "/home/ackerman/Workspace/OpenPCDet_ErrorModel/results.txt"

cfg_from_yaml_file(cfg_root, cfg)
logger = common_utils.create_logger(rank="Error")

class DemoDataset(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=True, root_path=None, logger=None, ext='.bin'):
        """
        Args:
            root_path:
            dataset_cfg:
            class_names:
            training:
            logger:
        """
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )
        self.root_path = root_path
        self.ext = ext
        data_file_list = glob.glob(str(root_path / f'*{self.ext}')) if self.root_path.is_dir() else [self.root_path]

        data_file_list.sort()
        self.sample_file_list = data_file_list

    def __len__(self):
        return len(self.sample_file_list)

    def __getitem__(self, index):
        if self.ext == '.bin':
            points = np.fromfile(self.sample_file_list[index], dtype=np.float32).reshape(-1, 4)
        elif self.ext == '.npy':
            points = np.load(self.sample_file_list[index])
        else:
            raise NotImplementedError

        input_dict = {
            'points': points,
            'frame_id': index,
        }

        data_dict = self.prepare_data(data_dict=input_dict)
        return data_dict

In [ ]:
demo_dataset = DemoDataset(
    dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False,
    root_path=Path(pcd_path), ext=".bin", logger=logger
)
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=demo_dataset)
model.load_params_from_file(filename=model_path, logger=logger, to_cpu=True)
model.cuda()
model.eval()

results_file = open(results_path, 'w')
results_file.write("Frame\t\tError_x\tError_y\tGT_x\tGT_y\tScore\n")
num_frames = 7481
num_car_labels = 0
num_car_predictions = 0

for i in range(num_frames):
    print(f"Processing {i}")
    pcd_path = os.path.join(data_path, "velodyne", f"{str(i).zfill(6)}.bin")
    point_cloud = np.fromfile(pcd_path, dtype=np.float32)  
    point_cloud = point_cloud.reshape(-1, 4)
    input_dict = {
        'points': point_cloud,
        'frame_id': i,
    }
    data_dict = demo_dataset.prepare_data(data_dict=input_dict)
    data_dict = demo_dataset.collate_batch([data_dict])
    load_data_to_gpu(data_dict)
    pred_dicts, _ = model.forward(data_dict)
    pred_boxes = pred_dicts[0]['pred_boxes']
    pred_scores = pred_dicts[0]['pred_scores']
    pred_labels = pred_dicts[0]['pred_labels']
    
    label_path = os.path.join(data_path, "label_2", f"{str(i).zfill(6)}.txt")
    car_positions = []
    with open(label_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip().split(" ")
            if line[0] == "Car":
                car_positions.append([float(line[11]), float(line[12]), float(line[13])])
    
    num_car_labels += len(car_positions)
    for car_position in car_positions:
        car_position = np.array([car_position[2], -car_position[0], -car_position[1]])
        for pred_box, pred_score, pred_label in zip(pred_boxes, pred_scores, pred_labels):
            pred_label = pred_label.cpu().detach().numpy()
            if pred_label == 1:
                pred_box = pred_box.cpu().detach().numpy()
                pred_score = pred_score.cpu().detach().numpy()
                error = (pred_box[:2] - car_position[:2])

                if np.linalg.norm(error) < 1:
                    results_file.write(f"{i}\t\t{error[0] - 0.25:.3f}\t{error[1]:.3f}\t{car_position[0]:.3f}\t{car_position[1]:.3f}\t{pred_score:.3f}\n")
                    num_car_predictions += 1

results_file.close()
print(f"Number of car labels: {num_car_labels}")
print(f"Number of car predictions: {num_car_predictions}")
print(f"Accuracy: {num_car_predictions / num_car_labels}")
